In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
tf.__version__


'2.4.1'

In [3]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

In [4]:
keras.__version__

'2.4.3'

In [5]:
rm -r "/content/train"

In [6]:
!unzip -q "/content/drive/My Drive/train.zip"

X

In [ ]:
!unzip -q "/content/drive/My Drive/test1.zip"

X

In [13]:
!mkdir -p train
!mkdir -p test1

X

In [ ]:
!cp -av -r "/content/drive/MyDrive/My data/train" "/content/train"

X

In [ ]:
%cp -av -r "/content/test1/" "/content/drive/My Drive/My data"

In [7]:
train_dir = '/content/train/'
test_dir = '/content/test1/'

In [8]:
import numpy as np 
import pandas as pd 

import shutil
from tqdm import tqdm

import os
from google.colab.patches import cv2_imshow
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

###ALL IMAGES ARE CHANGED HERE 

In [9]:
from PIL import Image
import glob
#image_list = []
d=0
for filename in glob.glob(train_dir+'*.jpg'): #assuming jpg
    img = cv2.imread(filename)
    #You can modify the code here#
    dst = np.empty_like(img) #create empty array the size of the image
    noise = cv2.randn(dst, (0,0,0), (20,20,20)) #add random img noise
    pup_noise = cv2.addWeighted(img, 0.5, noise, 0.5, 50) 

    # Blurring function; kernel=15, sigma=auto
    pup_blur = cv2.GaussianBlur(pup_noise, (15, 15), 0)
    #Your modification ends here#
    cv2.imwrite(filename, pup_blur)
    d+=1
    #if(d>1):
    #  break
print(d)

25000


In [10]:

if not os.path.exists(train_dir):
    os.mkdir(train_dir)
if not os.path.exists(train_dir+"dog"): 
    os.mkdir(train_dir+"dog")
if not os.path.exists(train_dir+"cat"):
    os.mkdir(train_dir+"cat")

In [11]:
import shutil
from tqdm import tqdm

In [12]:
#moving the picture into two subfolder according to their names.

for i in tqdm(os.listdir(train_dir)):
#     print(i)
    if i.split(".")[0] == "dog":
      if(os.path.isfile(os.path.join(train_dir,i))):
        shutil.move(os.path.join(train_dir,i),os.path.join(train_dir+"dog/",i))
    elif i.split(".")[0] == "cat":
      if(os.path.isfile(os.path.join(train_dir,i))):
        shutil.move(os.path.join(train_dir,i),os.path.join(train_dir+"cat/",i))

100%|██████████| 25002/25002 [00:01<00:00, 24340.53it/s]


In [14]:
if not os.path.exists(test_dir):
    os.mkdir(test_dir)
if not os.path.exists(test_dir+"dog"):
    os.mkdir(test_dir+"dog")
if not os.path.exists(test_dir+"cat"):
    os.mkdir(test_dir+"cat")

In [15]:
for i in tqdm(os.listdir(train_dir+"dog")[:3000]):
    shutil.move(os.path.join(train_dir+"dog",i),os.path.join(test_dir+"dog",i)) 
for i in tqdm(os.listdir(train_dir+"cat")[:3000]):
    shutil.move(os.path.join(train_dir+"cat",i),os.path.join(test_dir+"cat",i)) 

100%|██████████| 3000/3000 [00:00<00:00, 16923.02it/s]


In [16]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=train_dir,target_size=(224,224)) #chossing the folder not the files
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=test_dir, target_size=(224,224))

Found 19000 images belonging to 2 classes.
Found 7394 images belonging to 2 classes.


In [17]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

In [18]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [19]:
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [20]:
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs = vggmodel.inputs, outputs = predictions)

In [21]:
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [22]:
model_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=40, verbose=1, mode='auto')
model_final.fit_generator(generator= traindata, steps_per_epoch= 2, epochs= 100, validation_data= testdata, validation_steps=1, callbacks=[checkpoint,early])
model_final.save_weights("vgg16_1.h5")     

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
2/2 [==============================] - 55s 34s/step - loss: 1.2489 - accuracy: 0.5625 - val_loss: 0.9978 - val_accuracy: 0.5625
Epoch 2/100
2/2 [==============================] - 51s 34s/step - loss: 0.9552 - accuracy: 0.5104 - val_loss: 0.8151 - val_accuracy: 0.5312
Epoch 3/100
2/2 [==============================] - 52s 34s/step - loss: 0.8640 - accuracy: 0.5729 - val_loss: 0.9139 - val_accuracy: 0.4688
Epoch 4/100
2/2 [==============================] - 51s 34s/step - loss: 0.8718 - accuracy: 0.6562 - val_loss: 0.6278 - val_accuracy: 0.6875
Epoch 5/100
2/2 [==============================] - 51s 34s/step - loss: 0.7757 - accuracy: 0.6458 - val_loss: 0.6163 - val_accuracy: 0.7188
Epoch 6/100
2/2 [==============================] - 51s 34s/step - loss: 0.5462 - accuracy: 0.7292 - val_loss: 0.4941 - val_accuracy: 0.7500
Epoch 7/100
2/2 [==============================] - 51s 34s/step - loss: 0.5774 - accuracy: 0.6979 - val_loss: 0.4052 - val_accuracy: 0.8125
Epoch 8/100
2/2 [===